<a href="https://colab.research.google.com/github/brendanfitz/fitz-ai/blob/main/Personal_Voice_Memo_Transcriber.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from transformers import pipeline

pipe = pipeline('automatic-speech-recognition', model='openai/whisper-large-v3')

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json: 0.00B [00:00, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

...

normalizer.json: 0.00B [00:00, ?B/s]

added_tokens.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

preprocessor_config.json:   0%|          | 0.00/340 [00:00<?, ?B/s]

Device set to use cuda:0


In [ ]:
from pathlib import Path

audio_wd = Path('/content/drive/MyDrive/Voice Memos')

In [ ]:
for file_name in audio_wd.glob('*'):
  result = pipe(str(file_name))
  print(file_name.stem + '\n' + result['text'])

Green Leaf Ln 2
 Focus on extending your lower spine on the treadmill.


ValueError: Soundfile is either not in the correct format or is malformed. Ensure that the soundfile has a valid audio file extension (e.g. wav, flac or mp3) and is not corrupted. If reading from a remote URL, ensure that the URL is the full address to **download** the audio file.

In [ ]:
result['text']

' Focus on extending your lower spine on the treadmill.'

In [ ]:
str(file_name)

'/content/drive/MyDrive/Voice Memos/SW Bypass 2.m4a'

In [ ]:
!pip install librosa

In [ ]:
import librosa
import soundfile as sf

try:
    y, sr = librosa.load(str(file_name))
    print(f"Audio loaded successfully. Sample rate: {sr}, Number of samples: {len(y)}")
except Exception as e:
    print(f"Error loading audio file with librosa: {e}")
    # Attempt to load with soundfile as an alternative
    try:
        y, sr = sf.read(str(file_name))
        print(f"Audio loaded successfully with soundfile. Sample rate: {sr}, Number of samples: {len(y)}")
    except Exception as e_sf:
        print(f"Error loading audio file with soundfile: {e_sf}")
        print("Could not load the audio file with either librosa or soundfile.")